In [1]:
#####Probit based Hierarchical Matrix Factorization####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
import scipy.linalg
import scipy.stats as ss
import seaborn as sns
from numpy.random import *
from scipy import optimize
from scipy.stats import norm
from scipy import sparse

In [2]:
####任意の相関行列(分散共分散行列)を作成する関数####
##任意の相関行列を作る関数
def CorM(col, lower, upper, eigen_lower, eigen_upper):
    #相関行列の初期値を定義する
    cov_vec = (upper - lower) *rand(col*col) + lower   #相関係数の乱数ベクトルを作成
    rho = np.reshape(np.array(cov_vec), (col, col)) * np.tri(col)   #乱数ベクトルを下三角行列化
    Sigma = np.diag(np.diag(rho + rho.T) + 1) - (rho + rho.T)   #対角成分を1にする
    
    #相関行列を正定値行列に変更
    #固有値分解を実行
    eigen = scipy.linalg.eigh(Sigma)
    eigen_val = eigen[0] 
    eigen_vec = eigen[1]
    
    #固有値が負の数値を正にする
    for i in range(eigen_val.shape[0]-1):
        if eigen_val[i] < 0:
            eigen_val[i] = (eigen_upper - eigen_lower) * rand(1) + eigen_lower
            
    #新しい相関行列の定義と対角成分を1にする
    Sigma = np.dot(np.dot(eigen_vec, np.diag(eigen_val)), eigen_vec.T)
    normalization_factor = np.dot(pow(np.diag(Sigma), 0.5)[:, np.newaxis], pow(np.diag(Sigma), 0.5)[np.newaxis, :])
    Cor = Sigma / normalization_factor
    return Cor

In [3]:
##相関行列から分散共分散行列に変換する関数
def covmatrix(Cor, sigma_lower, sigma_upper):
    sigma = (sigma_upper - sigma_lower) * rand(np.diag(Cor).shape[0]) + sigma_lower
    sigma_factor = np.dot(sigma[:, np.newaxis], sigma[np.newaxis, :])
    Cov = Cor * sigma_factor
    return Cov

In [4]:
####データの発生####
##データの設定
k = 10   #基底数
hh = 10000   #ユーザー数
item = 3000   #アイテム数
pt = np.random.poisson(np.random.gamma(25, 1/0.3, hh), hh)   #購買接触数
hhpt = np.sum(pt)   #総サンプル数

In [5]:
#IDの設定
user_id = np.repeat(range(hh), pt)
pt_id = np.zeros(hhpt, dtype='int')
for i in range(hh):
    pt_id[user_id==i] = range(pt[i])

In [6]:
#インデックスの作成
index = np.array(range(hhpt))
user_index = [i for i in range(hh)]
for i in range(hh):
    user_index[i] = index[user_id==i]

In [15]:
##説明変数の生成
##素性ベクトルを生成
k1 = 2; k2 = 3; k3 = 4
x1 = np.array(np.random.random(hhpt*k1)).reshape((hhpt, k1))
x2 = np.zeros((hhpt, k2))
for j in range(k2):
    prob = np.random.uniform(0.25, 0.55, 1)
    x2[:, j] = np.random.binomial(1, prob, hhpt)
x3 = np.random.multinomial(1, np.random.dirichlet(np.repeat(3.0, k3), 1).reshape(k3), hhpt)
x3 = np.delete(x3, np.argmin(np.sum(x3, axis=0)), axis=1)   #冗長な変数の削除
X = np.concatenate((np.repeat(1, hhpt)[:, np.newaxis], x1, x2, x3), axis=1)
column = X.shape[1]

In [16]:
##階層モデルの説明変数を生成
#ユーザーの説明変数を生成
k1 = 1; k2 = 3; k3 = 5
u1 = np.array(np.random.random(hh*k1)).reshape((hh, k1))
u2 = np.zeros((hh, k2))
for j in range(k2):
    prob = np.random.uniform(0.25, 0.55, 1)
    u2[:, j] = np.random.binomial(1, prob, hh)
u3 = np.random.multinomial(1, np.random.dirichlet(np.repeat(3.0, k3), 1).reshape(k3), hh)
u3 = np.delete(u3, np.argmin(np.sum(u3, axis=0)), axis=1)   #冗長な変数の削除
u = np.concatenate((np.repeat(1, hh)[:, np.newaxis], u1, u2, u3), axis=1)
column_u = u.shape[1]

#アイテムの説明変数を生成
k1 = 2; k2 = 3; k3 = 4
v1 = np.array(np.random.random(item*k1)).reshape((item, k1))
v2 = np.zeros((item, k2))
for j in range(k2):
    prob = np.random.uniform(0.25, 0.55, 1)
    v2[:, j] = np.random.binomial(1, prob, item)
v3 = np.random.multinomial(1, np.random.dirichlet(np.repeat(3.0, k3), 1).reshape(k3), item)
v3 = np.delete(v3, np.argmin(np.sum(v3, axis=0)), axis=1)   #冗長な変数の削除
v = np.concatenate((np.repeat(1, item)[:, np.newaxis], v1, v2, v3), axis=1)
column_v = v.shape[1]

In [9]:
##アイテムの割当を生成
#セグメント割当を生成
topic = 25
phi = np.random.dirichlet(np.repeat(0.5, item), topic)
theta = np.random.dirichlet(np.repeat(2.5, topic), hh)
z = np.dot(np.array([np.random.multinomial(1, theta[i, :], 1) for i in range(hh)]).reshape(hh, topic), range(topic))

#多項分布からアイテムを生成
item_id = np.zeros(hhpt, dtype='int')
for i in range(hh):
    if i%1000==0:
        print(i)
    item_id[user_index[i]] = np.dot(np.random.multinomial(1, phi[z[i], :], pt[i]), range(item))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [10]:
#アイテムインデックスを作成
index = np.array(range(hhpt))
item_index = [j for j in range(item)]
for j in range(item):
    item_index[j] = index[item_id==i]

In [55]:
##階層モデルのパラメータの設定
#階層モデルの分散を設定
Cov = np.diag(uniform(0.01, 0.15, column))
Cov_u = np.diag(uniform(0.01, 0.2, k))
Cov_v = np.diag(uniform(0.01, 0.2, k))

#階層モデルの回帰係数を設定
alpha = np.hstack((np.random.normal(-0.2, 0.5, column_u).reshape(column_u, 1), 
                   np.random.normal(0, 0.5, (column-1)*column_u).reshape(column_u, column-1)))
alpha_u = np.random.normal(0, 0.5, k*column_u).reshape(column_u, k)
alpha_v = np.random.normal(0, 0.5, k*column_v).reshape(column_v, k)

##モデルパラメータを生成
#素性ベクトルと行列分解のパラメータを生成
beta = betat = np.dot(u, alpha) + np.random.multivariate_normal(np.repeat(0, column), Cov, hh)
theta_u = theta_ut = np.dot(u, alpha_u) + np.random.multivariate_normal(np.repeat(0, k), Cov_u, hh)
theta_v = theta_vt = np.dot(v, alpha_v) + np.random.multivariate_normal(np.repeat(0, k), Cov_v, item)

#標準偏差を設定
Sigma = 1

##潜在効用から応答変数を生成
#回帰モデルの平均構造
W = theta_u[user_id, :]
H = theta_v[item_id, :]
mu = np.dot(X * beta[user_id, :], np.repeat(1, column)) + np.dot(W * H, np.repeat(1, k))

#正規分布から潜在効用を生成
er = np.random.normal(0, Sigma, hhpt)   #モデル誤差
U = mu + er   #潜在効用
y = np.array((U > 0), dtype='int')   #潜在効用を購買ベクトルに変換

array([-1.02745504, -0.10019715, -1.26232179, ..., -3.86776498,
       -2.92406517, -2.87350849])

array([[ 1.27814671,  0.28999916, -0.12825995, ..., -0.10302746,
        -0.45532612,  0.5028598 ],
       [ 1.18529037,  0.60846914, -0.35506605, ...,  0.23045446,
        -0.10238381,  0.41229371],
       [-0.71189864,  0.52071086, -1.72019407, ...,  0.52400436,
         0.26242015, -0.92123632],
       ...,
       [-0.40350748,  1.05260487, -0.76665564, ...,  0.63390924,
         0.30666113, -0.3375893 ],
       [-0.61133537,  0.90402873, -2.01348149, ..., -0.30610234,
        -0.56908786, -0.12319796],
       [-1.05613259,  0.77685573, -1.7347576 , ..., -0.1218253 ,
         0.31808625, -0.46743125]])